In [ ]:
## QICK LIBRARY
from qick import *

## tProc Assembler
from qick.tprocv2_assembler import Assembler, Logger, LFSR

import numpy as np
import time

##### Load FGPA BitStream
soc = QickSoc('./qick_4x2_jac.bit')

In [ ]:
asm = """
//TEST program
// Write CORE_W_DT SFR
REG_WR s12 imm #12
REG_WR s13 imm #13

// Use External FLAG
REG_WR s_cfg imm cfg_flg_axi

REG_WR s_rand imm #0
REG_WR s13 imm #0

CHECK_FLAG:
   JUMP HERE -if(F)
   REG_WR s12 op -op(s_rand)
   REG_WR s13 op -op(s13 + #1)

   JUMP HERE -if(NF)
   REG_WR s12 op -op(s_rand)
   REG_WR s13 op -op(s13 + #1)
JUMP CHECK_FLAG

"""

p_txt, p_bin  = Assembler.str_asm2bin(asm)
soc.tproc.Load_PMEM(p_bin)
soc.tproc.clr_axi_flg()
soc.tproc.start()


soc.tproc.core_cfg  = 2
t_lfsr = LFSR()
t_lfsr.seed(0)
soc.tproc.read_sel  = 1
read_rand     = int(soc.tproc.axi_r_dt1)

check = 5
error = 0
print('Checking LFSR for '+str(2*check) +' counts')
for ind in range(check) :
    soc.tproc.set_axi_flg()
    expected_rand = int(t_lfsr.nxt())
    read_rand     = int(soc.tproc.axi_r_dt1)
    if (expected_rand != read_rand):
        error += 1
        print(expected_rand, read_rand, soc.tproc.axi_r_dt2)

    soc.tproc.clr_axi_flg()
    expected_rand =  t_lfsr.nxt()
    read_rand = soc.tproc.axi_r_dt1
    if (expected_rand != read_rand):
        error += 1
        print(expected_rand, read_rand, soc.tproc.axi_r_dt2)

if (error):
    print('Error in ', error, ' values')
else:
    print('All ', 2*check , ' LFSR values were OK')
